In [13]:
# Preprocesamiento simplificado de datos - Guía de Embarazo y Parto 2023
# Genera directamente los archivos finales necesarios

import PyPDF2
import re
import json
import os
from pathlib import Path
from datetime import datetime

# Configuración de rutas
raw_pdf_path = Path("raw/guia_embarazo_parto_2023.pdf")
clean_pdf_path = Path("processed/guia_embarazo_parto_clean.pdf")
clean_text_path = Path("processed/guia_embarazo_clean.txt")
chunks_path = Path("chunks/chunks_final.json")

# Crear directorios si no existen
clean_pdf_path.parent.mkdir(parents=True, exist_ok=True)
clean_text_path.parent.mkdir(parents=True, exist_ok=True)
chunks_path.parent.mkdir(parents=True, exist_ok=True)

print("=== PREPROCESAMIENTO SIMPLIFICADO ===")
print(f"📁 PDF original: {raw_pdf_path}")
print(f"📁 PDF limpio: {clean_pdf_path}")
print(f"📁 Texto limpio: {clean_text_path}")
print(f"📁 Chunks finales: {chunks_path}")
print()

# Verificar archivo original
if raw_pdf_path.exists():
    size_mb = raw_pdf_path.stat().st_size / (1024*1024)
    print(f"✅ Archivo original encontrado ({size_mb:.2f} MB)")
else:
    print(f"❌ Error: No se encuentra {raw_pdf_path}")
    print("   Coloca el archivo PDF original en la carpeta 'raw/'")

=== PREPROCESAMIENTO SIMPLIFICADO ===
📁 PDF original: raw\guia_embarazo_parto_2023.pdf
📁 PDF limpio: processed\guia_embarazo_parto_clean.pdf
📁 Texto limpio: processed\guia_embarazo_clean.txt
📁 Chunks finales: chunks\chunks_final.json

✅ Archivo original encontrado (3.54 MB)


In [14]:
def clean_pdf_direct(input_path, output_path):
    """
    Limpia el PDF directamente: 
    - Elimina primeras 123 páginas 
    - Elimina últimas 98 páginas
    - Elimina página 401 específicamente (RECOMENDACIONES PARA LA IMPLEMENTACIÓN)
    """
    try:
        with open(input_path, 'rb') as input_file:
            pdf_reader = PyPDF2.PdfReader(input_file)
            pdf_writer = PyPDF2.PdfWriter()
            
            total_pages = len(pdf_reader.pages)
            print(f"📄 Páginas originales: {total_pages}")
            
            # Calcular páginas a mantener
            start_page = 123  # Eliminar primeras 123 páginas (índice 0-122)
            end_page = total_pages - 98  # Eliminar últimas 98 páginas
            
            print(f"🗑️ Eliminando páginas 1-{start_page} (primeras)")
            print(f"🗑️ Eliminando páginas {end_page + 1}-{total_pages} (últimas)")
            
            # Eliminar página 401 específicamente
            # En el PDF original, después de eliminar las primeras 123 páginas,
            # la página 401 del visualizador será el índice 401-1=400 en base 0
            # Pero ajustado por eliminar las primeras 123, será 400-123=277
            exclude_page_adjusted = 401 - 123 - 1  # = 277 (índice 0-based después del ajuste)
            print(f"🗑️ Eliminando página 401 (RECOMENDACIONES) = índice ajustado {exclude_page_adjusted}")
            
            # Agregar páginas válidas al PDF limpio
            pages_added = 0
            for page_num in range(start_page, end_page):
                adjusted_index = page_num - start_page
                if adjusted_index != exclude_page_adjusted:
                    pdf_writer.add_page(pdf_reader.pages[page_num])
                    pages_added += 1
                else:
                    print(f"🗑️ Saltando página 401 (RECOMENDACIONES)")
            
            # Guardar PDF limpio
            with open(output_path, 'wb') as output_file:
                pdf_writer.write(output_file)
            
            final_size = output_path.stat().st_size / (1024*1024)
            print(f"✅ PDF limpio creado: {pages_added} páginas ({final_size:.2f} MB)")
            return True
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

In [ ]:
def extract_clean_text(pdf_path, output_path):
    """
    Extrae texto limpio del PDF en formato PÁGINA|CONTENIDO
    """
    try:
        import fitz  # PyMuPDF
        
        doc = fitz.open(str(pdf_path))
        total_pages = len(doc)
        print(f"📄 Extrayendo texto de {total_pages} páginas...")
        
        clean_lines = []
        
        for page_num in range(total_pages):
            page = doc.load_page(page_num)
            page_text = page.get_text()
            
            # Limpiar texto de la página
            clean_text = clean_page_content(page_text)
            
            # Solo agregar páginas con contenido válido
            if len(clean_text) > 30:
                line = f"{page_num + 1}|{clean_text}"
                clean_lines.append(line)
            
            if (page_num + 1) % 50 == 0:
                print(f"   ✓ {page_num + 1}/{total_pages} páginas procesadas")
        
        doc.close()
        
        # Guardar texto limpio
        with open(output_path, 'w', encoding='utf-8') as f:
            for line in clean_lines:
                f.write(line + '\n')
        
        file_size = output_path.stat().st_size / (1024*1024)
        print(f"✅ Texto limpio creado: {len(clean_lines)} páginas ({file_size:.2f} MB)")
        return len(clean_lines)
        
    except ImportError:
        print("❌ Error: Instala PyMuPDF con: pip install PyMuPDF")
        return 0
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return 0

def clean_page_content(text):
    """Limpia el contenido de una página"""
    if not text or len(text.strip()) < 5:
        return ""
    
    # Eliminar letras de recomendación aisladas (A, B, C, D, R, V) al inicio de líneas
    # Esto es específico para las tablas de recomendaciones
    text = re.sub(r'^\s*[A-DRV]\s+', '', text, flags=re.MULTILINE)
    
    # Normalizar espacios, pero manteniendo saltos de línea para el procesamiento anterior
    text = text.replace('\u00A0', ' ').replace('\u2028', ' ').replace('\u2029', ' ')
    
    # Eliminar marcadores de evidencia y símbolos
    # Formatos: 1++, 1+, 1-, 2++, 2+, 2-, 3, 4
    text = re.sub(r'\b\d[+\-]{1,2}\b', '', text)
    # Formatos: Ia, Ib, II, III, IV
    text = re.sub(r'\b(Ia|Ib|II|III|IV)\b', '', text)
    # Formatos numéricos y con cruces que quedaron
    text = re.sub(r'\+\d+\s*', '', text)
    text = re.sub(r'\b\d+\+\+?\b', '', text)
    text = re.sub(r'\b[IVX]+\+?\b', '', text)
    text = re.sub(r'[-_═─▬■|▌▐┃]{3,}', '', text)
    text = text.replace('√', '').replace('↑', '')
    
    # Eliminar números de página y referencias entre paréntesis
    text = re.sub(r'^\d{1,3}\s+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+\d{1,3}$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\(\d{1,4}\)', '', text)
    
    # Finalmente, colapsar todos los espacios múltiples a uno solo
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [16]:
def create_chunks(text_file_path, output_path, chunk_size=500, overlap=100):
    """
    Crea chunks directamente del archivo de texto limpio con section_number y section_title
    """
    # Definir secciones del documento
    SECTIONS = {
        1: {
            "title": "PREVENCIÓN Y DETECCIÓN TEMPRANA DE LAS ALTERACIONES DEL EMBARAZO",
            "start_page": 1,
            "end_page": 145
        },
        2: {
            "title": "ABORDAJE DE LAS COMPLICACIONES HIPERTENSIVAS ASOCIADAS AL EMBARAZO", 
            "start_page": 146,
            "end_page": 201
        },
        3: {
            "title": "INFECCIONES EN EL EMBARAZO: RUPTURA PREMATURA DE MEMBRANAS (RPM)",
            "start_page": 202,
            "end_page": 277
        },
        4: {
            "title": "NUTRICIÓN EN EL EMBARAZO",
            "start_page": 278,
            "end_page": 310
        },
        5: {
            "title": "DETECCIÓN TEMPRANA DE LAS ANOMALÍAS DURANTE EL TRABAJO DE PARTO, ATENCIÓN DEL PARTO NORMAL Y DISTÓCICO",
            "start_page": 311,
            "end_page": 357
        },
        6: {
            "title": "COMPLICACIONES HEMORRÁGICAS ASOCIADAS AL EMBARAZO",
            "start_page": 358,
            "end_page": 400
        }
    }
    
    def get_section_for_page(page_num):
        """Determina a qué sección pertenece una página"""
        for section_num, section_info in SECTIONS.items():
            if section_info["start_page"] <= page_num <= section_info["end_page"]:
                return section_num, section_info["title"]
        return 1, SECTIONS[1]["title"]  # Por defecto sección 1
    
    try:
        with open(text_file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        chunks_data = []
        
        for line in lines:
            line = line.strip()
            if not line or '|' not in line:
                continue
            
            page_num_str, content = line.split('|', 1)
            page_num = int(page_num_str.strip())
            content = content.strip()
            
            if len(content) < 30:
                continue
            
            # Obtener información de la sección
            section_number, section_title = get_section_for_page(page_num)
            
            # Crear chunks de esta página
            page_chunks = chunk_text_with_overlap(content, chunk_size, overlap)
            
            for chunk_idx, chunk_text in enumerate(page_chunks):
                chunk_data = {
                    "chunk_id": f"page{page_num}_chunk{chunk_idx + 1}",
                    "page_number": page_num,
                    "chunk_index": chunk_idx + 1,
                    "content": chunk_text,
                    "char_count": len(chunk_text),
                    "word_count": len(chunk_text.split()),
                    "source": "guia_embarazo_parto_clean.pdf",
                    "section_number": section_number,
                    "section_title": section_title
                }
                chunks_data.append(chunk_data)
        
        # Guardar chunks
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(chunks_data, f, ensure_ascii=False, indent=2)
        
        file_size = output_path.stat().st_size / (1024*1024)
        print(f"✅ Chunks creados: {len(chunks_data)} chunks ({file_size:.2f} MB)")
        return len(chunks_data)
        
    except Exception as e:
        print(f"❌ Error creando chunks: {str(e)}")
        return 0

def chunk_text_with_overlap(text, chunk_size=500, overlap=100):
    """Divide texto en chunks con overlap"""
    if len(text) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        
        if end >= len(text):
            chunk = text[start:]
            if chunk.strip():
                chunks.append(chunk.strip())
            break
        
        chunk = text[start:end]
        
        # Intentar terminar en punto o espacio
        if end < len(text) and text[end] not in [' ', '.', ',', ';']:
            last_sentence = max(chunk.rfind('.'), chunk.rfind('!'), chunk.rfind('?'))
            if last_sentence > chunk_size * 0.6:
                chunk = chunk[:last_sentence + 1]
                end = start + last_sentence + 1
            else:
                last_space = chunk.rfind(' ')
                if last_space > chunk_size * 0.7:
                    chunk = chunk[:last_space]
                    end = start + last_space
        
        chunks.append(chunk.strip())
        start = max(end - overlap, start + 1)
    
    return chunks

In [17]:
# PROCESO PRINCIPAL - Ejecuta todo de una vez
print("=== INICIANDO PREPROCESAMIENTO COMPLETO ===")
print()

if not raw_pdf_path.exists():
    print("❌ Error: Coloca el archivo PDF en la carpeta 'raw/'")
else:
    # Paso 1: Limpiar PDF
    print("🔄 PASO 1: Limpiando PDF...")
    success_pdf = clean_pdf_direct(raw_pdf_path, clean_pdf_path)
    
    if success_pdf:
        print("✅ PDF limpio creado correctamente")
        print()
        
        # Paso 2: Extraer texto limpio
        print("🔄 PASO 2: Extrayendo texto limpio...")
        pages_extracted = extract_clean_text(clean_pdf_path, clean_text_path)
        
        if pages_extracted > 0:
            print("✅ Texto limpio creado correctamente")
            print()
            
            # Paso 3: Crear chunks
            print("🔄 PASO 3: Creando chunks...")
            chunks_created = create_chunks(clean_text_path, chunks_path)
            
            if chunks_created > 0:
                print("✅ Chunks creados correctamente")
                print()
                print("🎉 ¡PREPROCESAMIENTO COMPLETADO!")
                print()
                print("📋 RESUMEN:")
                print(f"   📄 PDF limpio: {clean_pdf_path}")
                print(f"   📝 Texto limpio: {clean_text_path}")
                print(f"   🧩 Chunks: {chunks_path}")
                print(f"   📊 Páginas procesadas: {pages_extracted}")
                print(f"   📊 Chunks generados: {chunks_created}")
                print()
                print("🎯 Archivos listos para usar en RAG!")
            else:
                print("❌ Error creando chunks")
        else:
            print("❌ Error extrayendo texto")
    else:
        print("❌ Error limpiando PDF")

=== INICIANDO PREPROCESAMIENTO COMPLETO ===

🔄 PASO 1: Limpiando PDF...
📄 Páginas originales: 623
🗑️ Eliminando páginas 1-123 (primeras)
🗑️ Eliminando páginas 526-623 (últimas)
🗑️ Eliminando página 401 (RECOMENDACIONES) = índice ajustado 277
🗑️ Saltando página 401 (RECOMENDACIONES)
✅ PDF limpio creado: 401 páginas (1.95 MB)
✅ PDF limpio creado correctamente

🔄 PASO 2: Extrayendo texto limpio...
📄 Extrayendo texto de 401 páginas...
🗑️ Saltando página 401 (RECOMENDACIONES)
✅ PDF limpio creado: 401 páginas (1.95 MB)
✅ PDF limpio creado correctamente

🔄 PASO 2: Extrayendo texto limpio...
📄 Extrayendo texto de 401 páginas...
   ✓ 50/401 páginas procesadas
   ✓ 100/401 páginas procesadas
   ✓ 50/401 páginas procesadas
   ✓ 100/401 páginas procesadas
   ✓ 150/401 páginas procesadas
   ✓ 200/401 páginas procesadas
   ✓ 150/401 páginas procesadas
   ✓ 200/401 páginas procesadas
   ✓ 250/401 páginas procesadas
   ✓ 300/401 páginas procesadas
   ✓ 250/401 páginas procesadas
   ✓ 300/401 páginas p

In [18]:
# VERIFICACIÓN OPCIONAL - Revisar archivos generados
print("=== VERIFICACIÓN DE ARCHIVOS GENERADOS ===")
print()

# Verificar PDF limpio
if clean_pdf_path.exists():
    try:
        with open(clean_pdf_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            pages = len(pdf_reader.pages)
            size_mb = clean_pdf_path.stat().st_size / (1024*1024)
        print(f"📄 PDF limpio: {pages} páginas ({size_mb:.2f} MB)")
    except:
        print("❌ Error verificando PDF")
else:
    print("❌ PDF limpio no encontrado")

# Verificar texto limpio
if clean_text_path.exists():
    try:
        with open(clean_text_path, 'r', encoding='utf-8') as f:
            lines = len(f.readlines())
        size_mb = clean_text_path.stat().st_size / (1024*1024)
        print(f"📝 Texto limpio: {lines} páginas ({size_mb:.2f} MB)")
    except:
        print("❌ Error verificando texto")
else:
    print("❌ Texto limpio no encontrado")

# Verificar chunks
if chunks_path.exists():
    try:
        with open(chunks_path, 'r', encoding='utf-8') as f:
            chunks_data = json.load(f)
        size_mb = chunks_path.stat().st_size / (1024*1024)
        print(f"🧩 Chunks: {len(chunks_data)} chunks ({size_mb:.2f} MB)")
        
        if chunks_data:
            avg_chars = sum(c['char_count'] for c in chunks_data) // len(chunks_data)
            print(f"📊 Promedio caracteres por chunk: {avg_chars}")
            
            # Verificar solo section_number y section_title
            sample_chunk = chunks_data[0]
            if 'section_number' in sample_chunk and 'section_title' in sample_chunk:
                print("✅ Metadatos de sección incluidos:")
                print(f"   • section_number: {sample_chunk['section_number']}")
                print(f"   • section_title: {sample_chunk['section_title']}")
            else:
                print("❌ Metadatos de sección NO encontrados")
    except:
        print("❌ Error verificando chunks")
else:
    print("❌ Chunks no encontrados")

print()
print("✅ Verificación completada")

=== VERIFICACIÓN DE ARCHIVOS GENERADOS ===

📄 PDF limpio: 401 páginas (1.95 MB)
📝 Texto limpio: 401 páginas (0.78 MB)
🧩 Chunks: 2364 chunks (1.70 MB)
📊 Promedio caracteres por chunk: 422
✅ Metadatos de sección incluidos:
   • section_number: 1
   • section_title: PREVENCIÓN Y DETECCIÓN TEMPRANA DE LAS ALTERACIONES DEL EMBARAZO

✅ Verificación completada
